# Loop 5 Analysis: Understanding the Gap to Target

## Key Questions:
1. What is the actual gap between best public solution (70.676) and target (68.923)?
2. What techniques have NOT been tried yet?
3. What are the correct SA parameters from jiweiliu?

In [ ]:
import json
import numpy as np
import pandas as pd

# Load session state
with open('/home/code/session_state.json') as f:
    state = json.load(f)

print('=== EXPERIMENT SUMMARY ===')
for exp in state['experiments']:
    print(f"{exp['id']}: {exp['name']} - Score: {exp['score']:.6f}")

print('\n=== SUBMISSIONS ===')
for sub in state['submissions']:
    print(f"{sub['model_name']}: CV={sub['cv_score']:.6f}, LB={sub['lb_score']}")

print('\n=== KEY FINDINGS ===')
for finding in state['data_findings'][-5:]:
    print(f"- {finding['finding'][:200]}...")

In [ ]:
# Calculate the gap
best_score = 70.676102
target = 68.922808
gap = best_score - target
gap_pct = gap / target * 100

print(f'Best Score: {best_score:.6f}')
print(f'Target: {target:.6f}')
print(f'Gap: {gap:.6f} ({gap_pct:.2f}%)')
print()
print('This is a 2.5% improvement needed - significant but achievable.')

In [ ]:
# Analyze what the jiweiliu kernel ACTUALLY does vs what we tried
print('=== JIWEILIU SA PARAMETERS (CORRECT) ===')
jiweiliu_params = {
    'Tmax': 0.001,
    'Tmin': 0.000001,
    'nsteps': 10,
    'nsteps_per_T': 10000,
    'position_delta': 0.002,
    'angle_delta': 1.0,
    'angle_delta2': 1.0,
    'delta_t': 0.002,
}

print('\n=== OUR EXPERIMENT PARAMETERS (WRONG) ===')
our_params = {
    'Tmax': 0.1,
    'Tmin': 1e-6,
    'nsteps': 5000,
    'nsteps_per_T': 100,
    'position_delta': 0.1,
    'angle_delta': 10.0,
    'angle_delta2': 5.0,
    'delta_t': 0.05,
}

print('\n=== PARAMETER COMPARISON ===')
for key in jiweiliu_params:
    ratio = our_params[key] / jiweiliu_params[key]
    print(f'{key}: jiweiliu={jiweiliu_params[key]}, ours={our_params[key]}, ratio={ratio:.1f}x')

In [ ]:
# What approaches have been tried?
print('=== APPROACHES TRIED ===')
approaches = [
    ('001_baseline', 'Pre-optimized saspav solution', 'No improvement possible'),
    ('002_ensemble', 'Ensemble from multiple sources', 'saspav already best for all N'),
    ('003_lattice', 'Lattice construction + backward prop', 'Worse than baseline'),
    ('004_numba_sa', 'Numba backward propagation', '0 improvements'),
    ('005_full_sa', 'Full SA with WRONG parameters', 'Worse than baseline'),
]

for name, desc, result in approaches:
    print(f'{name}: {desc} -> {result}')

print('\n=== APPROACHES NOT YET TRIED ===')
not_tried = [
    'SA with CORRECT jiweiliu parameters (Tmax=0.001, position_delta=0.002)',
    'C++ tree_packer_v18 optimizer (from jonathanchan kernel)',
    'Parallel grid exploration (all viable grid sizes)',
    'Fractional translation optimization',
    'Extended grid with append_x/append_y',
]
for approach in not_tried:
    print(f'- {approach}')

In [ ]:
# Check what C++ optimizers are available
import os

print('=== AVAILABLE C++ OPTIMIZERS ===')
optimizers = [
    '/home/code/bbox3_advanced',
    '/home/code/tree_packer',
    '/home/code/tree_packer_v18',
]

for opt in optimizers:
    if os.path.exists(opt):
        size = os.path.getsize(opt)
        print(f'{opt}: {size/1024:.1f} KB')
    else:
        print(f'{opt}: NOT FOUND')

In [ ]:
# Key insight: The jiweiliu kernel claims ~0.15 improvement
# But we need 1.75 points improvement
# This means we need to:
# 1. Run SA with correct parameters
# 2. Explore ALL grid configurations (not just 4)
# 3. Use C++ optimizer for speed
# 4. Combine multiple techniques

print('=== IMPROVEMENT NEEDED ===')
print(f'Gap to target: {gap:.6f}')
print(f'jiweiliu claims: ~0.15 improvement')
print(f'Remaining gap after jiweiliu: {gap - 0.15:.6f}')
print()
print('This suggests we need MULTIPLE techniques combined:')